In [1]:
import numpy as np
import tensorflow as tf

In [6]:
import tensorflow_datasets as tfds

In [7]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info = True, as_supervised = True)

In [47]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

#As the MNIST dataset is a 2 tuple iterable, we extract the 2 tuples(train and target) in the 2 variables below. 
#Iter makes the validation_data iterable
#and next tells the system to go to the next iteration value for allocate the values to the 2 variables below
validation_train, validation_target = next(iter(validation_data))

MODEL

Outline the Model

In [48]:
input_size = 748
output_size = 10
hidden_layer_size = 175
#Lets take all hidden layers of the same size here

model = tf.keras.Sequential([
    #we need to flatten the input from a tensor(a picture) to a vector
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax'),
                            ])


Choose Optimizer and Loss Function

In [49]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

Training

In [50]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_train, validation_target), verbose =2)

Epoch 1/5
540/540 - 1s - loss: 0.2536 - accuracy: 0.9256 - val_loss: 0.1156 - val_accuracy: 0.9653
Epoch 2/5
540/540 - 1s - loss: 0.0978 - accuracy: 0.9704 - val_loss: 0.0824 - val_accuracy: 0.9757
Epoch 3/5
540/540 - 1s - loss: 0.0677 - accuracy: 0.9784 - val_loss: 0.0727 - val_accuracy: 0.9787
Epoch 4/5
540/540 - 2s - loss: 0.0530 - accuracy: 0.9828 - val_loss: 0.0517 - val_accuracy: 0.9847
Epoch 5/5
540/540 - 1s - loss: 0.0404 - accuracy: 0.9867 - val_loss: 0.0496 - val_accuracy: 0.9852


In [51]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 997us/step - loss: 0.0888 - accuracy: 0.9743
